In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import json
import time
import pydde as d

In [2]:
#Parameters
samplenum = 6
time_length = 60 #seconds
batch_size = 2
filenum = int(samplenum/batch_size)

In [3]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [4]:
#Sample targets
y_target = np.zeros((samplenum,3))
y_target[:,0] = np.random.rand(samplenum)*2
y_target[:,1] = 2-np.random.rand(samplenum)*2
#y_target[:,1] = 2
y_target[:,2] = np.random.rand(samplenum)*2
y_target[20:50, :]
y_target[:, 0].size

6

In [5]:
#Sample p
start_time = time.time()

for b in range(filenum):
    data = {}
    data['parameter'] = []
    data['y_target'] = []
    data['loss'] = []
    data['iterations'] = []
    data['gradnorm'] = []
    for i in range(batch_size):
        params = dyn.get_p(y_target[i,:], dyn.p_init)
        data['y_target'].append(list(y_target[i,:]))
        data['parameter'].append(list(params.ptraj))
        data['loss'].append(params.loss)
        data['iterations'].append(params.iterations)
        data['gradnorm'].append(params.grad)

    with open(f'../Data/Samples/data_20k_2x2/data_{b}.json', 'w') as outfile:
        json.dump(data, outfile)
    print(f'batch number {b+1} completed')
    print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed
    
#add description file
Description = {}
Description['Description'] = [{'samplenum': samplenum, 'samplesperfile': batch_size, 'time length': time_length, 'range from': [0, 2, 0], 'range to': [2, 0, 2]}]
with open('../Data/Samples/data_20k_2x2/Description.json', 'w') as outfile:
    json.dump(Description, outfile)

8.257719829674409e-05
8.125556101812652e-05
batch number 1 completed
8.257719829674409e-05
8.125556101812652e-05
batch number 2 completed
8.257719829674409e-05
8.125556101812652e-05
batch number 3 completed

Duration: 0.375 min


In [ ]:
import seaborn as sns
x = y_target[:, 0]
sns.distplot(x)
sns.distplot(x, bins=20, kde=False, rug=True)